In [70]:
import pandas as pd
import numpy as np
import psycopg2 as pg2
import pandas.io.sql as sqlio
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine
import io
from impulsify.src.get_clusters import get_table
plt.style.use('ggplot')
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [46]:
conn = pg2.connect(dbname='impulsify', user='postgres', host='localhost', port='5435')
sql = '''select flag_name
            , city
            , state
            , property_code
            , kind
            , guest_profile
            , location_type
            , props_under_mgmt
            --, pc1
            --, pc2
            --, kmeans
            , agg_ward
            , transaction_month
            , itemizer
            , category_name
            , description
            , sum(quantity) number_sold
            , sum(subtotal) dollars_sold

        from (

        SELECT li.store_product_id
            , li.cart_id
            , cat.itemizer
            , cat.name category_name
            , mpd.description
            , li.quantity
            , li.price
            , li.cost
            , li.vendor_id
            --, li.discount
            --, c.discount_amount
            --, c.tendered
            --, c.reason
            , li.subtotal
            , TO_CHAR(c.transaction_date :: DATE, 'yyyy-mm') AS transaction_month
            , c.store_id
            , c.property_id
            , c.property_store_id
            , p.*

        FROM line_items li

        LEFT Join store_products sp
            on li.store_product_id = sp.id

        LEFT join mpd
            on sp.upc = mpd.upc

        LEFT JOIN carts c
            ON li.cart_id = c.id

        LEFT JOIN categories AS cat
            ON cat.id = mpd.category_id

        INNER JOIN property_clusters p
            ON c.property_id = p.property_id

        WHERE c.status='complete' 
            AND c.payment_type IN ('cash','credit','room')
            AND TO_CHAR(c.transaction_date :: DATE, 'yyyy-mm') >= '2019-01'
            and c.discount_amount = 0
            --and cat.name = 'Beverage: Soda'

        ) A

        group by flag_name
            , city
            , state
            , property_code
            , kind
            , guest_profile
            , location_type
            , props_under_mgmt
            --, pc1
            --, pc2
            --, kmeans
            , agg_ward
            , transaction_month
            , itemizer
            , category_name
            , description

        ;'''
sold = sqlio.read_sql_query(sql, conn)
conn = None

In [62]:
def push_table(df, table, db = 'impulsify', user = 'postgres',
               host= 'localhost', port = '5435'):
    '''
    inputs:
        df = dataframe to push
        table = table name to write in database
        db = database
        user, host, port = database connection info
    
    outputs:
        writres table to DB. Overwrites if table already exists
    '''
    engine = create_engine('postgresql+psycopg2://'+user+':@'+host+':'+port+'/'+db)
    df.head(0).to_sql(table, engine, if_exists='replace',index=False) #truncates the table
    
    conn = engine.raw_connection()
    cur = conn.cursor()
    output = io.StringIO()
    df.to_csv(output, sep='|', header=False, index=False)
    output.seek(0)
    contents = output.getvalue()
    cur.copy_from(output, table, sep = '|', null="") # null values become ''
    conn.commit()

In [63]:
push_table(sold, 'product_category_recommender')

In [57]:
print(sold.shape)
sold.head()

(476419, 15)


,flag_name,city,state,property_code,kind,guest_profile,location_type,props_under_mgmt,agg_ward,transaction_month,itemizer,category_name,description,number_sold,dollars_sold
0,Avid,Oklahoma City,OK,OKCAM,Select Service,Equal Combination,Airport,4,1,2019-06,Beverage: Non-Alcohol,Beverage: Coffee,"\tDunkin' Donuts, Iced Coffee, French Vanilla,...",2,8.00
1,Avid,Oklahoma City,OK,OKCAM,Select Service,Equal Combination,Airport,4,1,2019-06,Beverage: Non-Alcohol,Beverage: Energy,"Red Bull Energy Drink, 8.4 Oz, Can",7,27.68
2,Avid,Oklahoma City,OK,OKCAM,Select Service,Equal Combination,Airport,4,1,2019-06,Beverage: Non-Alcohol,Beverage: Energy,"Red Bull, Sugar Free, 8.4 Oz, Can",1,4.00
3,Avid,Oklahoma City,OK,OKCAM,Select Service,Equal Combination,Airport,4,1,2019-06,Beverage: Non-Alcohol,Beverage: Juice / Tea,"Gold Peak, Sweetened Iced Tea, 18.5 Oz, Bottle",1,3.00
4,Avid,Oklahoma City,OK,OKCAM,Select Service,Equal Combination,Airport,4,1,2019-06,Beverage: Non-Alcohol,Beverage: Juice / Tea,"Gold Peak Unsweetened Tea, 18.5 Oz, Bottle",2,6.68


In [25]:
def sold_by_store(property_code, category, month):
    top_store = sold[['agg_ward','property_code','category_name','description', 'transaction_month', 'number_sold']][sold.number_sold > 0]. \
        groupby(['agg_ward','property_code','category_name', 'description', 'transaction_month'], as_index=False).sum(). \
        sort_values(by=['agg_ward','property_code','category_name', 'transaction_month', 'number_sold'], ascending=False)    
    
    out = top_store[(top_store.property_code == property_code) & 
                    (top_store.category_name == category) & 
                    (top_store.transaction_month == month)]
    total_sold = np.sum(out.number_sold)
    out.loc[:, 'pct_of_sold'] = out['number_sold']/total_sold
    out.loc[:,'cum_pct'] = out['pct_of_sold'].cumsum()
    
    return out

def top_sold_by_cluster(cluster, category, month, num = 5):
    top_cluster = sold[['agg_ward','category_name','description', 'transaction_month','number_sold']][sold.number_sold > 0]. \
        groupby(['agg_ward','category_name', 'description', 'transaction_month'], as_index=False).sum(). \
        sort_values(by=['agg_ward','category_name', 'transaction_month', 'number_sold'], ascending=False)

    return top_cluster[(top_cluster.agg_ward == cluster) & 
                       (top_cluster.category_name == category) & 
                       (top_cluster.transaction_month == month)].head(num)

def top_sold_overall(category, month, num = 5):
    top_overall = sold[['category_name', 'description', 'transaction_month','number_sold']][sold.number_sold > 0]. \
        groupby(['category_name', 'description', 'transaction_month'], as_index=False).sum(). \
        sort_values(by=['category_name', 'transaction_month', 'number_sold'], ascending=False)
    
    return top_overall[(top_overall.category_name == category) & 
                       (top_overall.transaction_month == month)].head(num)

In [26]:
top_store.head()

,agg_ward,property_code,category_name,description,transaction_month,number_sold
2368,5,SPICC,Beverage: Soda,"Coca-Cola, Classic Soda, 20 Oz, Bottle",2019-01,90
2369,5,SPICC,Beverage: Soda,"Coca-Cola, Diet Soda, 20 Oz, Bottle",2019-01,68
2378,5,SPICC,Beverage: Soda,"Sprite, Lemon Lime, 20 Oz, Bottle",2019-01,51
2367,5,SPICC,Beverage: Soda,"Coca-Cola, Cherry Coca-Cola, 20 Oz, Bottle",2019-01,38
2377,5,SPICC,Beverage: Soda,"Pibb, Xtra, 20 Oz, Bottle",2019-01,32


In [27]:
sold_by_store('SPICC', 'Beverage: Soda', '2019-09')

,agg_ward,property_code,category_name,description,transaction_month,number_sold,pct_of_sold,cum_pct
4089,5,SPICC,Beverage: Soda,"Coca-Cola, Diet Soda, 20 Oz, Bottle",2019-09,55,0.283505,0.283505
4088,5,SPICC,Beverage: Soda,"Coca-Cola, Classic Soda, 20 Oz, Bottle",2019-09,34,0.175258,0.458763
4091,5,SPICC,Beverage: Soda,"Coca-Cola, Zero, 20 Oz, Bottle",2019-09,18,0.092784,0.551546
4087,5,SPICC,Beverage: Soda,"Coca-Cola, Cherry Coca-Cola, 20 Oz, Bottle",2019-09,17,0.087629,0.639175
4099,5,SPICC,Beverage: Soda,"Sprite, Lemon Lime, 20 Oz, Bottle",2019-09,15,0.077320,0.716495
4086,5,SPICC,Beverage: Soda,"Barq's, Root Beer, 20 Oz, Bottle",2019-09,10,0.051546,0.768041
4098,5,SPICC,Beverage: Soda,"Pibb, Xtra, 20 Oz, Bottle",2019-09,10,0.051546,0.819588
4100,5,SPICC,Beverage: Soda,"Sprite, Lemon-Lime, Sugar Free, 20 Oz, Bottle",2019-09,9,0.046392,0.865979
4094,5,SPICC,Beverage: Soda,"Fanta, Orange, 20 Oz, Bottle",2019-09,6,0.030928,0.896907
4095,5,SPICC,Beverage: Soda,"Fanta, Strawberry, 20 Oz, Bottle",2019-09,5,0.025773,0.922680


In [28]:
top_sold_by_cluster(1, 'Beverage: Soda', '2019-09')

,agg_ward,category_name,description,transaction_month,number_sold
285,1,Beverage: Soda,"Coca-Cola, Classic Soda, 20 Oz, Bottle",2019-09,6002
291,1,Beverage: Soda,"Coca-Cola, Diet Soda, 20 Oz, Bottle",2019-09,4821
476,1,Beverage: Soda,"Sprite, Lemon Lime, 20 Oz, Bottle",2019-09,2475
343,1,Beverage: Soda,"Dr. Pepper, 20 Oz, Bottle",2019-09,1280
307,1,Beverage: Soda,"Coca-Cola, Zero, 20 Oz, Bottle",2019-09,1237


In [29]:
top_sold_overall('Beverage: Soda', '2019-09')

,category_name,description,transaction_month,number_sold
87,Beverage: Soda,"Coca-Cola, Classic Soda, 20 Oz, Bottle",2019-09,19323
95,Beverage: Soda,"Coca-Cola, Diet Soda, 20 Oz, Bottle",2019-09,14152
388,Beverage: Soda,"Sprite, Lemon Lime, 20 Oz, Bottle",2019-09,7356
115,Beverage: Soda,"Coca-Cola, Zero, 20 Oz, Bottle",2019-09,4006
190,Beverage: Soda,"Dr. Pepper, 20 Oz, Bottle",2019-09,3665


In [41]:
def compare_products(property_code, category, month, num = 5):
    '''
    inputs: property_code
            category = product_category
            month = transaction_month
            num = number of products to compare
    outputs: print outs of top selling products in that category by this store vs. 
            store clusters and nationaly. As well as suggestions for different products
            in that category to stock and products to discontinue
    '''
    
    
    top_store_prods = sold_by_store(property_code, category, month).head(num)
    store_prods = sold_by_store(property_code, category, month)
    clust = np.max(sold[(sold.property_code == property_code) &
                        (sold.category_name == category)].agg_ward) # the max doesn't do anything all the clusters are the same
    clust_prods = top_sold_by_cluster(clust, category, month, num)
    tot_clust = np.sum(clust_prods.number_sold)
    natl_prods = top_sold_overall(category, month)
    tot_natl = np.sum(natl_prods.number_sold)
    
    desc = sold[['flag_name','city','state']][sold.property_code == property_code].iloc[0]
    print('For store #{0}, {1}, {2}, {3} {4}'.format(property_code,
                                                     desc.flag_name,
                                                     desc.city,
                                                     desc.state,
                                                     month))
    
    if store_prods.shape[0] == 0:
        print('No products sold in {}'.format(category))
        
        if natl_prods.shape[0] < num:
            n = natl_prods.shape[0]
        else:
            n = num
        print('\nNational top products in {}:'.format(category))
        for i in range(n):
            print('{0}. {1}: {2}%'.format(i+1, natl_prods.description.iloc[i],
                                    round(100 * natl_prods.number_sold.iloc[i]/tot_natl,1)))
        return print('')
    
    if sold_by_store(property_code, category, month).shape[0] < num:
        n = sold_by_store(property_code, category, month).shape[0]
    else:
        n = num
    print('\nYour top products in {}:'.format(category))
    for i in range(n):
        print('{0}. {1}: {2} units, {3}%'.format(i+1, top_store_prods.description.iloc[i],
                                          top_store_prods.number_sold.iloc[i], 
                                                 round(100 * top_store_prods.pct_of_sold.iloc[i],1)))
    
    if clust_prods.shape[0] < num:
        n = clust_prods.shape[0]
    else:
        n = num
    print('\nCluster({}) top products:'.format(clust))
    for i in range(n):
        print('{0}. {1}: {2}%'.format(i+1, clust_prods.description.iloc[i],
                                round(100 * clust_prods.number_sold.iloc[i]/tot_clust,1)))
    
    if natl_prods.shape[0] < num:
        n = natl_prods.shape[0]
    else:
        n = num
    print('\nNational top product:')
    for i in range(n):
        print('{0}. {1}: {2}%'.format(i+1, natl_prods.description.iloc[i],
                                round(100 * natl_prods.number_sold.iloc[i]/tot_natl,1)))
    
    '''
    products suggested for removal are not in top 5 of cluster and are in the
    bottom 10% cumulative of sales
    '''
    
    #change this to products that aren't in the top 10, not top 5
    to_remove = store_prods[(~store_prods.description.isin(clust_prods.description.unique())) &
                           (store_prods.cum_pct > .9)]
    
    if set(clust_prods.description) - set(store_prods.description) == set():
        print('\nYou are selling the top products already!')
        if to_remove.shape[0] > 0:
            print('\nConsider discontinue stocking:')
            for i in range(to_remove.shape[0]):
                print('{0}. {1}: {2} units, {3}%'.format(i+1, to_remove.description.iloc[i],
                                                        to_remove.number_sold.iloc[i],
                                                        round(100 * to_remove.pct_of_sold.iloc[i],1))) 
    else:
        print('\nStocking suggestions:')
        add = list(set(clust_prods.description) - set(store_prods.description))
        for idx, item in enumerate(add):
            print('{0}. {1}'.format(idx+1, item))
        if to_remove.shape[0] > 0:
            print('\nConsider discontinue stocking:')
            for i in range(to_remove.shape[0]):
                print('{0}. {1}: {2} units, {3}%'.format(i+1, to_remove.description.iloc[i],
                                                        to_remove.number_sold.iloc[i],
                                                        round(100 * to_remove.pct_of_sold.iloc[i],1)))


In [44]:
compare_products('YXEAP', 'Beverage: Soda', '2019-09', 5)

For store #YXEAP, Hampton Inn and Suites, Saskatoon, SK 2019-09

Your top products in Beverage: Soda:
1. Coca-Cola, Diet Coke, 12 Oz, Bottle : 6 units, 31.6%
2. A&W, Root Beer, 16.9 Oz, Bottle: 3 units, 15.8%
3. Fresca, 12 Oz, Can: 3 units, 15.8%
4. Fanta, Orange, 16 Oz, Bottle: 2 units, 10.5%
5. Jones, Orange & Cream, Cane Sugar Soda, 12 Oz, Bottle : 2 units, 10.5%

Cluster(0) top products:
1. Coca-Cola, Classic Soda, 20 Oz, Bottle: 41.7%
2. Coca-Cola, Diet Soda, 20 Oz, Bottle: 28.9%
3. Sprite, Lemon Lime, 20 Oz, Bottle: 14.7%
4. Coca-Cola, Zero, 20 Oz, Bottle: 8.4%
5. Dr. Pepper, 20 Oz, Bottle: 6.4%

National top product:
1. Coca-Cola, Classic Soda, 20 Oz, Bottle: 39.8%
2. Coca-Cola, Diet Soda, 20 Oz, Bottle: 29.2%
3. Sprite, Lemon Lime, 20 Oz, Bottle: 15.2%
4. Coca-Cola, Zero, 20 Oz, Bottle: 8.3%
5. Dr. Pepper, 20 Oz, Bottle: 7.6%

Stocking suggestions:
1. Dr. Pepper, 20 Oz, Bottle
2. Coca-Cola, Diet Soda, 20 Oz, Bottle
3. Coca-Cola, Zero, 20 Oz, Bottle
4. Coca-Cola, Classic Soda, 2

In [11]:
sold.category_name.unique()

array(['Beverage: Coffee', 'Beverage: Energy', 'Beverage: Juice / Tea',
       'Beverage: Milk', 'Beverage: Protein', 'Beverage: Soda',
       'Beverage: Sports Drink', 'Beverage: Water', 'Candy', 'Chips',
       'Cookies', 'Crackers', 'Frozen: Ice Cream',
       'Frozen: Non Ice Cream', 'Grocery', 'Healthy Snacks', 'Quick Meal',
       'Refrigerated Foods', 'Salty: Non-Cracker',
       'Sundries: Electronics', 'Sundries: OTC',
       'Sundries: Personal Care', 'Beverage: Specialty Water',
       'Beverage: Beer', 'Breakfast', 'Gum/Mints', 'Unknown', 'Souvenir',
       'Other', 'Beverage: Other', 'Sundries: Travel', 'Beverage: Liquor',
       'Gourmet', 'Gifts', 'Beverage: Wine',
       'Refrigerated Foods (Prepared In House)', 'Regional Foods',
       'Toys/Games', 'Hhonors', 'Sundries: Office', 'Seasonal',
       'Clothing: General', None, 'Clothing: Souvenir',
       'Greeting Cards/Books/Movies', 'Jewelry ', 'Newspapers/Magazines',
       'Souvenir/Clothing', 'Clothing: Sports', 'C

In [43]:
sold.property_code.unique()

array(['OKCAM', 'OKCAV', 'TULAV', '43183', 'IHG - CAEAT', 'FCLCW',
       'MGMBC', 'IHG - MQVCW', 'IHG - PHXBC', 'TN363', 'FL094',
       'IHG-ATLAT', 'IHG-ATLCP', 'IHG-ATLPW', 'IHG-BALWB', 'STLAP',
       'CLTCP', 'IHG - CMHOC', 'IHG-DENDT', 'YFCQS', 'GSPRM', 'WASHV',
       'MKCWS', 'PHLVF', 'LANCP', 'MIAAP', 'IHG - SFOMP', 'MKETH',
       'YQMMS', 'IHG - CHSSC', 'MCOOA', 'SFOCA', 'SATTD', 'SEADT',
       'SPICC', 'VNTCA', 'MKEMR', 'HPNCP', 'ALBSA:Gift Shop', 'ANPDT',
       'BOSCO', 'BOSSB', 'SFOAO', 'LGBCH', 'DFWCN', 'RAPDW', 'INDWW',
       'HPRLP', 'LGBMY', 'LAXMA', 'MCOAT', 'SLCWS', 'SANCE', 'FLLSM',
       'YTOCS', 'LAXHD', 'WASCR', 'SEABL', 'MSPAP', 'MSPWE', 'CVGBA',
       'SANLJ', 'CLTTB', 'DALLF', 'LRDES', 'DTTLI', 'CHILM', 'SEALW',
       'MEMPE', 'MIASP', 'YULDN', 'MCOOR', 'SATES', 'PDXBV', 'WASCC',
       'WASAL', 'WASFB', 'AVLTU', 'ERIAS', 'BTRDN', 'WBUGB', 'LAXBK',
       'CRWSO', 'CHALM', 'CHIMC', 'STLGA', 'MCOSW', 'DULMN', 'EUFHX',
       'AVLAP', 'RDUCL', 'HDSNC', '

In [41]:
sold[sold.itemizer == 'Food'].category_name.unique()

array(['Candy', 'Chips', 'Cookies', 'Crackers', 'Frozen: Ice Cream',
       'Frozen: Non Ice Cream', 'Grocery', 'Healthy Snacks', 'Quick Meal',
       'Refrigerated Foods', 'Salty: Non-Cracker', 'Breakfast',
       'Gum/Mints', 'Unknown', 'Other', 'Gourmet',
       'Refrigerated Foods (Prepared In House)', 'Regional Foods',
       'Hhonors', 'Dips/Salsa'], dtype=object)

send clusters by property code

make by month range instead of specific month

add retail price to recommender

query by brand what is top sellers

property code instead of store id

figure out products sold all categories, then figure out what goes in the cooler

'Quick Meal', 'Regional Foods', 'Refrigerated Foods (Prepared In House)' go in cooler

average 25 products worth of facing
need multiple facings if more than 7 in a shift

soda
water
energy
coffee
breakfast milk
breakfast drink
juice

balance sales volume with categorization/variety

variety of categories, e.g. breakfast beverage might not be in there at all

1. cooler
2. freezer
3. shelf stable food
4. shelf stable non-food (sundried need more assortment of categories)

In [71]:
list(get_table('SELECT DISTINCT transaction_month FROM product_category_recommender').iloc[:,0])

['2019-02',
 '2019-01',
 '2019-07',
 '2019-05',
 '2019-08',
 '2019-10',
 '2019-06',
 '2019-03',
 '2019-09',
 '2019-04']

In [72]:
sold.iloc[0,0]

'Avid'